In [ ]:
import sys
import os

# Get the absolute path of the parent directory of the current script
parent_dir = os.path.dirname(os.getcwd())

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from meg_qc.calculation.meg_qc_pipeline import make_derivative_meg_qc

config_file_path = parent_dir+'/meg_qc/settings/settings.ini' 
internal_config_file_path=parent_dir+'/meg_qc/settings/settings_internal.ini' # internal settings in in
#raw, raw_cropped_filtered_resampled, QC_derivs, QC_simple, df_head_pos, head_pos, scores_muscle_all1, scores_muscle_all2, scores_muscle_all3, raw1, raw2, raw3, avg_ecg, avg_eog = make_derivative_meg_qc(config_file_path, internal_config_file_path)

for_report = make_derivative_meg_qc(config_file_path, internal_config_file_path)

In [ ]:
entities =  {'subject': {'009'}, 'session': {'1'}, 'task': {'deduction'}, 'run': {1}, 'description': {'STDs', 'Muscle', 'ECGchannel', 'PSDwavesGrad', 'PtPsManual', 'Sensors', 'SimpleMetrics', 'RawInfo', 'PSDs', 'ReportStrings', 'PSDnoiseGrad', 'PSDwavesMag', 'ECGs', 'EOGchannel', 'PSDnoiseMag', 'EOGs'}}
print('entities: ', entities)


def modify_entity_name(entities):

    """
    Modify the names of the entities to be comply with ancpbids.

    Parameters
    ----------
    entities : dict
        A dictionary of entities and their subcategories.
    
    Returns
    -------
    dict
        A dictionary of entities and their subcategories with modified names
    """

    print('___MEGqc___: ', 'Entities found in the dataset: ', entities)

    #old_new_categories = {'description': 'METRIC', 'subject': 'SUBJECT', 'session': 'SESSION', 'task': 'TASK', 'run': 'RUN'}

    old_new_categories = {'description': 'METRIC'}

    categories_copy = entities.copy()

    for category, subcategories in categories_copy.items():
        # Convert the set of subcategories to a sorted list
        sorted_subcategories = sorted(subcategories, key=str)
        # If the category is in old_new_categories, replace it with the new category
        if category in old_new_categories: 
            #This here is to replace the old names with new like desc -> METRIC
            #Normally we d use it only for the METRIC, but left this way in case the principle will extend to other categories
            #see old_new_categories above.

            new_category = old_new_categories[category]
            entities[new_category] = entities.pop(category)
            # Replace the original set of subcategories with the modified list
            sorted_subcategories.insert(0, '_ALL_'+new_category+'S_')
            entities[new_category] = sorted_subcategories
        else: #if we dont want to rename categories
            sorted_subcategories.insert(0, '_ALL_'+category+'s_')
            entities[category] = sorted_subcategories

    #From METRIC remove whatever is not metric. 
    #Cos METRIC is originally a desc entity which can contain just anything:
            
    if 'METRIC' in entities:
        valid_metrics = ['_ALL_METRICS_', 'STDs', 'PSDs', 'PtPsManual', 'PtPsAuto', 'ECGs', 'EOGs', 'Head', 'Muscle']
        entities['METRIC'] = [x for x in entities['METRIC'] if x.lower() in [metric.lower() for metric in valid_metrics]]

    return entities

entities = modify_entity_name(entities)
print('___MEGqc___: ', 'Entities after modification: ', entities)

In [ ]:
entities = {
    'subject': {'009'},
    'session': {'1'},
    'task': {'deduction'},
    'run': {1},
    'description': {'STDs', 'Muscle', 'ECGchannel', 'PSDwavesGrad', 'PtPsManual', 'Sensors', 'SimpleMetrics', 'RawInfo', 'PSDs', 'ReportStrings', 'PSDnoiseGrad', 'PSDwavesMag', 'ECGs', 'EOGchannel', 'PSDnoiseMag', 'EOGs'}
}
print('entities: ', entities)

def modify_entity_name(entities):
    # Create a copy of entities
    categories = entities.copy()

    # Rename 'description' to 'METRIC' and sort the values
    categories = {
        ('METRIC' if k == 'description' else k): sorted(v, key=str)
        for k, v in categories.items()
    }

    #From METRIC remove whatever is not metric. 
    #Cos METRIC is originally a desc entity which can contain just anything:
                
    if 'METRIC' in categories:
        valid_metrics = ['_ALL_METRICS_', 'STDs', 'PSDs', 'PtPsManual', 'PtPsAuto', 'ECGs', 'EOGs', 'Head', 'Muscle']
        categories['METRIC'] = [x for x in categories['METRIC'] if x.lower() in [metric.lower() for metric in valid_metrics]]

    #add '_ALL_' to the beginning of the list for each category:

    for category, subcategories in categories.items():
        categories[category] = ['_ALL_'+category+'s_'] + subcategories

    return categories

print('categories: ', categories)

In [ ]:
import mne
import re

# Load the CTF dataset
data = '/Volumes/SSD_DATA/MEG_data/CTF/ds000246/sub-0001/meg/sub-0001_task-AEF_run-01_meg.ds'
data = '/Volumes/SSD_DATA/MEG_data/CTF/ds000247/sub-0003/ses-01/meg/sub-0003_ses-01_task-rest_run-01_meg.ds'
data = '/Volumes/SSD_DATA/MEG_data/CTF/ds002761/sub-311/meg/sub-311_task-loc_run-01_meg.ds'
raw = mne.io.read_raw_ctf(data, preload=True)


#data = '/Volumes/SSD_DATA/MEG_data/openneuro/ds003483/sub-009/ses-1/meg/sub-009_ses-1_task-deduction_run-1_meg.fif'
#raw = mne.io.read_raw_fif(data, preload=True)

print(raw.filenames)

# Loop through each channel and print its type and unit
# for nch, ch in enumerate(raw.info['chs']):
#     ch_name = ch['ch_name']
#     ch_type = mne.channel_type(raw.info, nch)  # Correct function to get channel type
#     ch_unit = ch['unit']  # Get the unit code
#     print(f"Channel: {ch_name}, Type: {ch_type}, Unit: {ch_unit}")

picked_channels = mne.pick_types(raw.info, meg='mag')
for ch in picked_channels:
    ch_name = raw.info['ch_names'][ch]
    print(f"Channel: {ch_name}")
    ch_unit_code = raw.info['chs'][ch]['unit']
    print(str(ch_unit_code))
    ch_unit_str = str(ch_unit_code)
    #find str after 'UNIT_' in ch_unit_code:
    match = re.search(r'UNIT_(\w)', ch_unit_str)
    if match:
        unit = match.group(1)
    else:
        unit = 'unknown'
    print(f"Unit: {unit}")



In [ ]:
#raw.set_channel_types({'1': 'grad', '2': 'mag', '3': 'mag'})

ch = 31
ch_name = raw.info['ch_names'][ch]
print(f"Channel: {ch_name}")
print(raw.info['chs'][ch]['kind'])
print('ch type: ', mne.channel_type(raw.info, ch))
ch_unit_code = raw.info['chs'][ch]['unit']
print(str(ch_unit_code))

In [ ]:
import copy
raw2 = copy.deepcopy(raw)
ch2 = 32
raw2.set_channel_types({'MLC12-2910': 'grad'})

ch_name = raw2.info['ch_names'][ch]
print(f"Channel: {ch_name}")
print(raw2.info['chs'][ch]['kind'])
print('ch type: ', mne.channel_type(raw2.info, ch))
ch_unit_code = raw2.info['chs'][ch]['unit']
print(str(ch_unit_code))

In [ ]:
import mne

# Load MEG CTF data
data = '/Volumes/SSD_DATA/MEG_data/CTF/ds000246/sub-0001/meg/sub-0001_task-AEF_run-01_meg.ds'
raw = mne.io.read_raw_ctf(data, preload=True)

# Get channel information
info = raw.info

# Check if a channel is mag or grad
mags = mne.pick_types(info, meg='mag')
grads = mne.pick_types(info, meg='grad')

print(f'Magnetometer channels: {len(mags)}')
print(f'Gradiometer channels: {len(grads)}')

#print gradiometer channels with names and unit:
mag_channels = [[info['ch_names'][ch], info['chs'][ch]['unit'], info['chs'][ch]['kind']] for ch in mags]
grad_channels = [[info['ch_names'][ch], info['chs'][ch]['unit'], info['chs'][ch]['kind']] for ch in grads]
print('MAGS', mag_channels)
print('GRADS', grad_channels)


In [ ]:
print(mne.__version__)

In [ ]:
info['chs'][56]


In [ ]:
info['chs'][57]

In [ ]:
info

In [ ]:
import mne

info = raw.info

# Step 2: Save the info object to a .fif file
mne.io.write_info('info.fif', info)

# Step 3: Load the info object from the file when needed
info_loaded = mne.io.read_info('info.fif')

# Step 4: Create an MNE Report
report = mne.Report(title='MEG Report with Info')

# Step 5: Convert info to a string format for HTML embedding
info_html = info_loaded._repr_html_()


# Step 6: Add the info to the report as an HTML block
report.add_html(info_html, 'Info Object')

# Step 7: Save the report to an HTML file
report.save('meg_report_with_info.html', overwrite=True)


In [ ]:
# Changing Channel Kind and Units

from mne.io.constants import FIFF
import copy

raw_new = copy.deepcopy(raw)

# Get the info structure containing channel info
info = raw_new.info

# Function to convert all magnetometer channels to gradiometers
for ch in info['chs']:
    # If the channel is a magnetometer
    print('ch["kind"]', ch['kind'])
    if ch['kind'] == FIFF.FIFFV_MEG_MAG:
        # Change the channel type to gradiometer
        ch['kind'] = FIFF.FIFFV_MEG_GRAD
        # Change the unit to Tesla per meter (T/m)
        ch['unit'] = FIFF.FIFF_UNIT_T_M

# Optionally, check that the channels were updated correctly
for ch in info['chs']:
    if ch['kind'] == FIFF.FIFFV_MEG_GRAD:
        print(f"Channel {ch['ch_name']} is now a gradiometer with unit {mne.io.constants.FIFF.unit_dict[ch['unit']]}.")

# Save the modified data (if needed)
raw_new.save('modified_meg.fif', overwrite=True)

In [ ]:
print(raw.info)

In [ ]:
#ancpbids issues:
import ancpbids

dataset_path = '/Volumes/SSD_DATA/MEG_QC_stuff/data/openneuro/ds003483' #ds000117'
dataset = ancpbids.load_dataset(dataset_path)
entities = dataset.query_entities(scope='derivatives')

print(entities)

In [ ]:
import mne
import matplotlib.pyplot as plt

# Load CTF data
data = '/Volumes/SSD_DATA/MEG_data/CTF/ds000246/sub-0001/meg/sub-0001_task-AEF_run-01_meg.ds'
raw = mne.io.read_raw_ctf(data, preload=True)

# Access the info structure
info = raw.info

# Print the channel names
print("All channel names:")
print(info['ch_names'])

# Get the types of channels
picks_grad = data.copy().pick('grad').ch_names if 'grad' in data else []

# Check the number of gradiometers
n_gradiometers = sum([1 for t in picks_grad if t == 'grad'])
print(f"Number of gradiometers: {n_gradiometers}")




In [ ]:
file_name_in_path = "sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg"
file_name_in_obj = "sub-01_ses-meg_task-facerecognition_run-01_meg"

if file_name_in_obj not in file_name_in_path:
    raise ValueError('Different names in list_of_files and entities_per_file')

In [ ]:

import ancpbids

dataset_path = '/Volumes/SSD_DATA/MEG_data/CTF/ds000246'
sid = '0001'
dataset = ancpbids.load_dataset(dataset_path)

artifacts = dataset.query(suffix="meg", return_type="object", subj=sid)
# convert to folders of found files
folders = map(lambda a: a.get_parent().get_absolute_path(), artifacts)
# remove duplicates
folders = set(folders)
# convert to liust before filtering
folders = list(folders)

# filter for folders which end with ".ds" (including os specific path separator)
# folders = list(filter(lambda f: f.endswith(f"{os.sep}.ds"), folders))

# Filter for folders which end with ".ds"
filtered_folders = [f for f in folders if f.endswith('.ds')]

print(filtered_folders)

entities_per_file = dataset.query(sub=sid, suffix='meg', scope='raw')
print(entities_per_file)

entities_per_file_res4 = dataset.query(sub=sid, suffix='meg', extension='.res4')

print(entities_per_file_res4)

print(entities_per_file_res4[0].name)

#In entities_per_file filer out only thouse who have unique name, disregard the extension in this name:
#Return entities_per_file_filtered as list of objects

entities_per_file_filtered = []

for entity in entities_per_file:
    name = entity.name.split('.')[0]
    if name not in entities_per_file_filtered:
        entities_per_file_filtered.append(entity)

print(entities_per_file_filtered)

In [ ]:
import ancpbids
import os

dataset_path = '/Users/jenya/Local Storage/Job Uni Rieger lab/data/ds83'
sid = '009'
dataset = ancpbids.load_dataset(dataset_path)

list_of_files = sorted(list(dataset.query(suffix='meg', extension='.fif', return_type='filename', subj=sid)))
entities_per_file = dataset.query(sub=sid, suffix='meg', extension='.fif')
# sort list_of_sub_jsons by name key to get same order as list_of_files
entities_per_file = sorted(entities_per_file, key=lambda k: k['name'])

#check we got same number of jsons and files:
if len(list_of_files) != len(entities_per_file):
    raise ValueError('Different number of files and jsons')

print(list_of_files)
print(entities_per_file)

In [ ]:
artifacts = dataset.query(suffix="meg", return_type="object", subj=sid)

print(artifacts)
# convert to folders of found files
folders = map(lambda a: a.get_parent().get_absolute_path(), artifacts)
# remove duplicates
folders = set(folders)
# convert to liust before filtering
folders = list(folders)

print(folders)

In [ ]:
schema = dataset.get_schema()
derivative = dataset.create_derivative(name="Meg_QC")
derivative.dataset_description.GeneratedBy.Name = "MEG QC Pipeline"
subject_folder = derivative.create_folder(type_=schema.Subject, name='sub-'+sid)
calculation_folder = subject_folder.create_folder(name='calculation')

meg_artifact = calculation_folder.create_artifact(raw=entities_per_file_res4[0])
#meg_artifact = calculation_folder.create_artifact(raw=entities_per_file[0])

print(meg_artifact.get_absolute_path())

meg_artifact.add_entity('desc', 'TEST') #file name
meg_artifact.suffix = 'meg'
meg_artifact.extension = '.txt'
meg_artifact.content = 'dummy text'

ancpbids.write_derivative(dataset, derivative) 

In [ ]:
import mne

file_path = '/Volumes/SSD_DATA/MEG_data/CTF/ds000246/sub-0001/meg/sub-0001_task-AEF_run-01_meg.ds'
raw = mne.io.read_raw_ctf(file_path, preload=True)


In [ ]:
import ancpbids

metric = 'test'

dataset = ancpbids.load_dataset('/Volumes/SSD_DATA/camcan')
entities = dataset.query_entities()

#for sub in chosen_entities['subject']:
sub = 'CC220697'

entities_per_file = dataset.query(sub=sub, suffix='meg', extension='.fif')
print('___MEGqc___: list_of_sub_jsons', entities_per_file)
sub_json = entities_per_file[0]
print('___MEGqc___: sub_json', sub_json)


#Want: get entities of sub jason: sub, ses, run, task.
# find corresponding tsv file with same entities and use it to create report.
# save report with the same entities

schema = dataset.get_schema()

derivative = dataset.create_derivative(name="Meg_QC")
derivative.dataset_description.GeneratedBy.Name = "MEG QC Pipeline"
subject_folder = derivative.create_folder(type_=schema.Subject, name='sub-'+sub)
#strings = dataset.query(suffix='meg', extension='.json', return_type='filename', subj=sub, ses = chosen_entities['session'], task = chosen_entities['task'], run = chosen_entities['run'], desc = 'ReportStrings', scope='derivatives')
strings = dataset.query(suffix='meg', extension='.json', return_type='filename', subj=sub, desc = 'ReportStrings', scope='derivatives')
print(strings)

# Now prepare the derivative to be written:
meg_artifact = subject_folder.create_artifact(raw=sub_json)
meg_artifact.add_entity('desc', metric) #file name
meg_artifact.suffix = 'meg'
meg_artifact.extension = '.html'


#0. Get entities of each of tsvs_to_plot, save them into a dictionary with the tsv path as key and entities as values.
#1. Get all sub jsons and their entities, save into a dictionary with the json path as key and entities as values.
#2. Find the tsvs that have the same entities as the jsons.
#3. Make a dict: key = json, value = tsvs.


#OR:
#get all tsv files for 1 metric
#match what sub json (before '_meg') were they created for by just matching string up until '_desc'


strings = dataset.query(suffix='meg', extension='.json', return_type='filename', subj=sub, desc = 'ReportStrings', scope='derivatives')


2.



#2. Loop: get entities of 1 json(raw).

    #3. Match the entities of the json with the entities of the tsvs.

    #4. If match: create a derivative from these tsvs.

    #5. Save this derivative with the entities of the json.

#6. Repeat for all jsons.

In [ ]:
import re

sub_json = {'name': 'sub-CC220697_ses-passive_task-passive_desc-ECGs', 'extension': '.fif', 'suffix': 'meg'}

# match = re.search(r'sub-(\w+)(_ses-(\w+))?(_task-(\w+))?(_run-(\w+))?', sub_json['name'])
# match = re.search(r'sub-(\w+)(?:_ses-(\w+))?(?:_task-(\w+))?(?:_run-(\w+))?', sub_json['name'])
# match = re.search(r'sub-(\w+)_ses-(\w+)_task-(\w+)_run-(\w+)', sub_json['name'])
match = re.search(r'sub-(\w+)_ses-(\w+)_task-(\w+)_desc-(\w+)', sub_json['name'])

print(match.group(1))
print(match.group(2))
print(match.group(3))
print(match.group(4))



In [ ]:
json_entitities = []
for entity in ['sub', 'ses', 'task', 'run']:
    if entity in sub_json['name']:
        json_entitities.append(entity)

#now combine json_entitities in a string and find values for them in the json file name:
#then compare them with the values of the tsv file name:

match_str = ''
for entity in json_entitities:
    match_str += entity + '-(\w+)_'
#cut the underscore at the end:
match_str = match_str[:-1]
#shouldd look like:
#match_str = 'sub-(\w+)_ses-(\w+)_task-(\w+)_run-(\w+)'


match = re.search(match_str, sub_json['name'])

#print all existing groups:
for i in range(1, len(match.groups())+1):
    print(match.group(i))

In [ ]:
import numpy as np
from scipy.stats import pearsonr

def normalize_wave(wave):
    mean = np.mean(wave)
    std = np.std(wave)
    return (wave - mean) / std

def compute_shape_correlation(wave1, wave2):
    # Pearson correlation yielding a value between -1 and 1
    correlation, _ = pearsonr(wave1, wave2)
    return correlation

def compute_mean_amplitude(wave):
    # Calculate the mean amplitude of the wave
    return np.mean(np.abs(wave))

def normalize_amplitudes(amplitudes):
    # Normalize amplitudes to a range of 0 to 1
    min_amp = np.min(amplitudes)
    max_amp = np.max(amplitudes)
    return (amplitudes - min_amp) / (max_amp - min_amp)

def compute_combined_score(shape_similarity, normalized_amplitude, alpha=0.5, beta=0.5):
    return alpha * shape_similarity + beta * normalized_amplitude

# Example usage
reference_wave = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
waves_to_compare = [
    np.array([2, 4, 6, 8, 10, 12, 14, 16, 18, 20]),  # Similar shape, higher amplitude
    np.array([1, 2, 3, 3, 5, 6, 7, 7, 9, 10]),      # Similar shape, varied amplitude
    np.array([10, 9, 8, 7, 6, 5, 4, 3, 2, 1]),      # Different shape, similar amplitude
    np.array([0, 0, 1, 0, 0, 0, 0, 0, 5, 0])        # Flat wave, different shape and amplitude
]

# Normalize the reference wave
normalized_ref_wave = normalize_wave(reference_wave)

print('normalized_ref_wave:',normalized_ref_wave)


# Calculate shape similarity and amplitudes
shape_similarities = []
amplitudes = []

for wave in waves_to_compare:
    # Normalize the target wave
    normalized_wave = normalize_wave(wave)
    print('normalized_wave:',normalized_wave)
    
    # Compute shape similarity
    shape_similarity = compute_shape_correlation(normalized_ref_wave, normalized_wave)
    shape_similarities.append(shape_similarity)
    
    # Compute mean amplitude (we're using the original waves for amplitude)
    amplitude = compute_mean_amplitude(wave)
    amplitudes.append(amplitude)

# Normalize the amplitudes to a [0, 1] range
normalized_amplitudes = normalize_amplitudes(np.array(amplitudes))

# Compute combined scores
combined_scores = []

for shape_similarity, normalized_amplitude in zip(shape_similarities, normalized_amplitudes):
    score = compute_combined_score(shape_similarity, normalized_amplitude, alpha=0.5, beta=0.5)
    combined_scores.append(score)

# Rank the waves based on the combined score
ranked_indices = np.argsort(combined_scores)[::-1]  # Sort in descending order

# Display the results
for index in ranked_indices:
    print(f"Shape similarity: {shape_similarities[index]:.2f}, Normalized amplitude: {normalized_amplitudes[index]:.2f}, Combined score: {combined_scores[index]:.2f}")


#Plot with plotly the waves and the refernce wave:

import plotly.graph_objects as go

fig = go.Figure()

# Add the reference wave
fig.add_trace(go.Scatter(x=np.arange(len(reference_wave)), y=reference_wave, mode='lines', name='Reference wave'))

# Add the waves to compare
for i, wave in enumerate(waves_to_compare):
    fig.add_trace(go.Scatter(x=np.arange(len(wave)), y=wave, mode='lines', name=f'Wave {i + 1}'))

fig.update_layout(title='Comparison of Waves',
                    xaxis_title='Time',
                    yaxis_title='Amplitude')

fig.show()



In [ ]:
#Checking the ECG correlation algorythm here:

import numpy as np
#read tsv file into a data frame:
import pandas as pd
df = pd.read_csv('/Users/jenya/Local Storage/Job Uni Rieger lab/data/ds83/derivatives/Meg_QC/sub-009/sub-009_ses-1_task-deduction_run-1_desc-ECGs_meg.tsv', sep='\t')

metric = 'ECG'
ecg_eog_scores = ['ecg_corr_coeff', 'ecg_amplitude_ratio', 'ecg_similarity_score']
add_scores = all(column_name in df.columns and not df[column_name].empty and df[column_name].notnull().any() for column_name in ecg_eog_scores)
print(add_scores)

for index, row in df.iterrows():
    print(row['Name'])
    print(row[ecg_eog_scores[0]])
    print(row[ecg_eog_scores[1]])
    print(row[ecg_eog_scores[2]])
    customdata=np.array([row[ecg_eog_scores[0]], row[ecg_eog_scores[1]], row[ecg_eog_scores[2]]])
    print(customdata)

df_sorted = df.reindex(df['ecg_corr_coeff'].abs().sort_values(ascending=False).index)

metric='ecg'
#print out the name of the channels and the corr_coeff of this channel in the sorted list:
#for index, row in df_sorted.iterrows():
#    print('______', row['Name'], row[metric.lower()+'_corr_coeff'])


df_ref = pd.read_csv('/Users/jenya/Local Storage/Job Uni Rieger lab/data/ds83/derivatives/Meg_QC/sub-009/sub-009_ses-1_task-deduction_run-1_desc-ECGchannel_meg.tsv', sep='\t')

#make several plots: each contains 1 channel and the reference channel mean_rwave_shifted

#refernce: mean_rwave_shifted column in df_ref:
ch_data_ref = df_ref['mean_rwave_shifted']
#remove nans:
ch_data_ref = ch_data_ref.dropna()
#Scale the data from -1 to 1:
ch_data_ref = (ch_data_ref - ch_data_ref.min())/(ch_data_ref.max()-ch_data_ref.min())*2-1

#time:
t = df_ref['mean_rwave_time']

#Channels:
col_prefix = 'smoothed_mean_ecg_sec_'

chs_vals = {}
chs_vals_scaled = {}
ch_to_plot = ['MEG1531', 'MEG1131', 'MEG1541', 'MEG0111', 'MEG1031', 'MEG0621']
for ch in ch_to_plot:
    ch_data = []
    for index, row in df_sorted.iterrows():
        if row['Name'] == ch: #plot only mag/grad
            for col in df.columns:
                if col_prefix in col:
                    #ch_data = row[col] #or maybe 
                    ch_data.append(row[col])

    #Scale ch_data list from -1 to 1:
    min_val = min(ch_data)
    max_val = max(ch_data)
    
    ch_data_scaled = [(2*(x - min_val) / (max_val - min_val)) - 1 for x in ch_data]

    chs_vals[ch] = ch_data
    chs_vals_scaled[ch] = ch_data_scaled


# for each channel calculate pearson correlation p value with the reference channel:
corr_vals = {}
corr_vals_scaled = {}
from scipy.stats import pearsonr
for ch in ch_to_plot:
    corr, p_val = pearsonr(ch_data_ref, chs_vals[ch])
    corr_scaled, p_val_scaled = pearsonr(ch_data_ref, chs_vals_scaled[ch])
    #save to corr_vals:
    #round corr to 3 decimal places:
    corr = round(corr, 3)
    corr_vals[ch] = corr
    corr_scaled = round(corr_scaled, 3)
    corr_vals_scaled[ch] = corr_scaled

print('corr_vals:', corr_vals)
print('corr_vals_scaled', corr_vals_scaled)


# make plots with plotly:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=3, cols=2, subplot_titles=('MEG1531 (most)'+', corr: '+str(corr_vals['MEG1531']), 'MEG1131 (most)'+', corr: '+str(corr_vals['MEG1131']), 'MEG1541 (mid)'+', corr: '+str(corr_vals['MEG1541']), 'MEG0111 (mid)'+', corr: '+str(corr_vals['MEG0111']), 'MEG1031 (least)'+', corr: '+str(corr_vals['MEG1031']), 'MEG0621 (least)'+', corr: '+str(corr_vals['MEG0621'])))

for i, ch in enumerate(ch_to_plot):
    fig.add_trace(go.Scatter(x=t, y=ch_data_ref, mode='lines', name='ref', line=dict(color='black')), row=i//2+1, col=i%2+1)
    fig.add_trace(go.Scatter (x=t, y=chs_vals_scaled[ch], mode='lines', name=ch), row=i//2+1, col=i%2+1)
    #for each channel add their corr value into the subplot title:


fig.update_layout(title_text='ECG channels')
#make subplot titles smaller:
# Update the font size of the subplot titles
for i in fig['layout']['annotations']:
    i['font'] = dict(size=10)


fig.show()

#Now plot all 6 channels in 1 plot with scaling:
fig = go.Figure()
for ch in ch_to_plot:
    fig.add_trace(go.Scatter(x=t, y=chs_vals_scaled[ch], mode='lines', name=ch+': '+str(corr_vals_scaled[ch])))

fig.show()

#and same without scaling:
fig = go.Figure()
for ch in ch_to_plot:
    fig.add_trace(go.Scatter(x=t, y=chs_vals[ch], mode='lines', name=ch+': '+str(corr_vals[ch])))

fig.show()

#Look especially at 1131: it has very small amplitude, but the correlation is very high. 
#Because pearson cares only about the shape, not amplitude. so it basically sees the scaled version of each channel.
#and in the scaled version this channel is very well correlated.
#Note: disregard the corr sign, we care about abs corr value, cos signe is just defined by 
#the placement of the electrode on the head against the magnetic field

#So again, we should think of how to include the amplitude into the correlation calculation.
#And maybe define some corre value at which the channels with be separated into most/middle/least affected,
#not just equally devide into 3 groups.
# For example here 4 channels have very similar strong correlation(0.7-0.6), but they r divided into 2 most and lest.


In [ ]:
#p;laying with hovertemplate here:


import random
import numpy as np  

def plot_df_of_channels_data_as_lines_by_lobe_csv(f_path: str, metric: str, x_values, m_or_g, df=None):

    """
    Plots data from a data frame as lines, each lobe has own color.
    Data is taken from previously saved tsv file.

    Parameters
    ----------
    f_path : str
        Path to the csv file with the data to plot.
    metric : str
        The metric of the data to plot: 'psd', 'ecg', 'eog', 'smoothed_ecg', 'smoothed_eog'.
    
    Returns
    -------
    fig : plotly.graph_objects.Figure
        Plotly figure.

    """
    if f_path is not None:
        df = pd.read_csv(f_path, sep='\t') #TODO: maybe remove reading csv and pass directly the df here?
    else:
        df = df


    fig = go.Figure()
    traces_lobes=[]
    traces_chs=[]

    add_scores = False #in most cases except ecg/eog we dont add scores to the plot
    if metric.lower() == 'psd':
        col_prefix = 'PSD_Hz_'
    elif metric.lower() == 'ecg':
        col_prefix = 'mean_ecg_sec_'
    elif metric.lower() == 'eog':
        col_prefix = 'mean_eog_sec_'
    elif metric.lower() == 'smoothed_ecg' or metric.lower() == 'ecg_smoothed':
        col_prefix = 'smoothed_mean_ecg_sec_'
        #Need to check if all 3 columns exist in df, are not empty and are not none - if so, add scores to hovertemplate:
        ecg_eog_scores = ['ecg_corr_coeff', 'ecg_amplitude_ratio', 'ecg_similarity_score']
        add_scores = all(column_name in df.columns and not df[column_name].empty and df[column_name].notnull().any() for column_name in ecg_eog_scores)
    elif metric.lower() == 'smoothed_eog' or metric.lower() == 'eog_smoothed':
        col_prefix = 'smoothed_mean_eog_sec_'
        #Need to check if all 3 columns exist in df, are not empty and are not none - if so, add scores to hovertemplate:
        ecg_eog_scores = ['eog_corr_coeff', 'eog_amplitude_ratio', 'eog_similarity_score']
        add_scores = all(column_name in df.columns and not df[column_name].empty and df[column_name].notnull().any() for column_name in ecg_eog_scores)
    else:
        print('No proper column in df! Check the metric!')

   
    for index, row in df.iterrows():

        if row['Type'] == m_or_g: #plot only mag/grad
            ch_data = []
            for col in df.columns:
                if col_prefix in col:

                    #ch_data = row[col] #or maybe 
                    ch_data.append(row[col])

                    # normally color must be same for all channels in lobe, so we could assign it before the loop as the color of the first channel,
                    # but here it is done explicitly for every channel so that if there is any color error in chs_by_lobe, it will be visible
            
            color = row['Lobe Color']

            #traces_chs += [go.Scatter(x=x_values, y=ch_data, line=dict(color=color), name=row['Name'] , legendgroup=row['Lobe'] , legendgrouptitle=dict(text=row['Lobe'].upper(), font=dict(color=color)))]

            if add_scores:
                
                traces_chs += [go.Scatter(
                    x=x_values, 
                    y=ch_data, 
                    line=dict(color=color), 
                    name=row['Name'],
                    legendgroup=row['Lobe'],
                    legendgrouptitle=dict(text=row['Lobe'].upper(), font=dict(color=color)),
                    hovertemplate = (
                    '<b>'+row['Name']+'</b><br>' +
                    'time: %{x}, magnitude: %{y}<br>' +
                    '<i>corr_coeff: </i>'+'{:.2f}'.format(row[ecg_eog_scores[0]])+'<br>' +
                    '<i>amplitude_ratio: </i>'+'{:.2f}'.format(row[ecg_eog_scores[1]])+'<br>' +
                    '<i>similarity_score: </i>'+'{:.2f}'.format(row[ecg_eog_scores[2]])+'<br>'
                ))]
            else:
                print('_________NO SCORES TO ADD TO HOVER TEMPLATE_________')
                traces_chs += [go.Scatter(
                    x=x_values, 
                    y=ch_data, 
                    line=dict(color=color), 
                    name=row['Name'],
                    legendgroup=row['Lobe'],
                    legendgrouptitle=dict(text=row['Lobe'].upper(), font=dict(color=color))
                )]
            #legendgrouptitle is group tile on the plot. legendgroup is not visible on the plot - it s used for sorting the legend items in update_layout() below.

                # hovertemplate=
                #     '<b>%{y}</b><br>' +
                #     row['Name']+'<br>' +
                #     '<i>corr_coeff: </i>%{customdata[0]}<br>' +
                #     '<i>amplitude_ratio: </i>%{customdata[1]}<br>' +
                #     '<i>similarity_score: </i>%{customdata[2]}<br>',
                # customdata=np.array([row[ecg_eog_scores[0]], row[ecg_eog_scores[1]], row[ecg_eog_scores[2]]]),
                
    # sort traces in random order:
    # When you plot traves right away in the order of the lobes, all the traces of one color lay on top of each other and yu can't see them all.
    # This is why they are not plotted in the loop. So we sort them in random order, so that traces of different colors are mixed.
    traces = traces_lobes + sorted(traces_chs, key=lambda x: random.random())

    downsampling_factor = 1  # replace with your desired downsampling factor
    # Create a new list for the downsampled traces
    traces_downsampled = []

    # Go through each trace
    for trace in traces:
        # Downsample the x and y values of the trace
        x_downsampled = trace['x'][::downsampling_factor]
        y_downsampled = trace['y'][::downsampling_factor]

        # Create a new trace with the downsampled values
        trace_downsampled = go.Scatter(x=x_downsampled, y=y_downsampled, line=trace['line'], name=trace['name'], legendgroup=trace['legendgroup'], legendgrouptitle=trace['legendgrouptitle'])

        # Add the downsampled trace to the list
        traces_downsampled.append(trace_downsampled)


    # Now first add these traces to the figure and only after that update the layout to make sure that the legend is grouped by lobe.
    fig = go.Figure(data=traces)

    fig.update_layout(legend_traceorder='grouped', legend_tracegroupgap=12, legend_groupclick='toggleitem')
    #You can make it so when you click on lobe title or any channel in lobe you activate/hide all related channels if u set legend_groupclick='togglegroup'.
    #But then you cant see individual channels, it turn on/off the whole group. There is no option to tun group off by clicking on group title. Grup title and group items behave the same.

    #to see the legend: there is really nothing to sort here. The legend is sorted by default by the order of the traces in the figure. The onl way is to group the traces by lobe.
    #print(fig['layout'])

    #https://plotly.com/python/reference/?_ga=2.140286640.2070772584.1683497503-1784993506.1683497503#layout-legend-traceorder
    

    return fig


metric = 'ecg_smoothed'
f_path = '/Users/jenya/Local Storage/Job Uni Rieger lab/data/ds83/derivatives/Meg_QC/sub-009/sub-009_ses-1_task-deduction_run-1_desc-ECGs_meg.tsv'
f_path2 = '/Users/jenya/Local Storage/Job Uni Rieger lab/data/ds83/derivatives/Meg_QC/sub-009/sub-009_ses-1_task-deduction_run-1_desc-ECGchannel_meg.tsv'
m_or_g = 'mag'

df = pd.read_csv(f_path, sep='\t')
df2 = pd.read_csv(f_path2, sep='\t')

x_values = df2['mean_rwave_time']
fig = plot_df_of_channels_data_as_lines_by_lobe_csv(f_path, metric, x_values, m_or_g, df)
fig.show()

In [ ]:
#New idea: combine amplitude and correlation:

import numpy as np

#Calculate amplitude ratio for each channel:

# Function to calculate the Root Mean Square (RMS) amplitude
def rms_amplitude(wave):
    return np.sqrt(np.mean(np.square(wave)))

# RMS amplitude of all comparison waves excluding the reference_wave
rms_all_comp_waves = np.mean([rms_amplitude(wave) for wave in chs_vals.values()])

amplitude_ratios = {}
for ch in ch_to_plot:
    #amplitude_ratio = rms_amplitude(chs_vals[ch]) / rms_amplitude(ch_data_ref)
    amplitude_ratio = rms_amplitude(chs_vals[ch]) / rms_all_comp_waves  

    amplitude_ratios[ch] = amplitude_ratio

print('amplitude_ratios:', amplitude_ratios)
print('corr_vals_scaled:', corr_vals_scaled)    


# Combine the two metrics like: similarity_score = correlation * amplitude_ratio

similarity_scores = {}
for ch in ch_to_plot:
    similarity_score = abs(corr_vals_scaled[ch]) * abs(amplitude_ratios[ch])
    similarity_scores[ch] = similarity_score

print('similarity_scores:', similarity_scores)

#Now plot all 6 channels in 1 plot with similarity scores:
fig = go.Figure()
for ch in ch_to_plot:
    fig.add_trace(go.Scatter(x=t, y=chs_vals[ch], mode='lines', name=ch+': '+str(similarity_scores[ch])))
fig.show()


#ONE MORE idea for later: 
# scale all channels not just from-1 to 1 but rescale proportional to max amplitude of all channels:

#Calculate max amplitude of all channels:
max_amplitude = max([max(chs_vals[ch]) for ch in ch_to_plot])
min_amplitude = min([min(chs_vals[ch]) for ch in ch_to_plot])

#Scale all channels:
chs_vals_scaled_max = {}
for ch in ch_to_plot:
    ch_data_scaled_max = [(2*(x - min_amplitude) / (max_amplitude - min_amplitude)) - 1 for x in chs_vals[ch]]
    chs_vals_scaled_max[ch] = ch_data_scaled_max

#Now plot all 6 channels in 1 plot with scaling:


In [ ]:
import numpy as np

# Empty list
empty_list = []
print(len(empty_list))  # Outputs: 0

# Empty numpy array
empty_array = np.array([])
print(len(empty_array))  # Outputs: 0

empty_list = empty_list if len(empty_list) else print('Empty list is empty')
empty_array = empty_array if len(empty_array) else print('Empty numpy array is empty')

In [ ]:
import pandas as pd
print(pd.__version__)

In [ ]:
import mne
raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds003483/sub-022/ses-1/meg/sub-022_ses-1_task-deduction_run-1_meg.fif', on_split_missing='ignore')

#print(raw.info)
print(type(raw.info))

#How to extract raw.info from data file, save in derivs and later embed it into the mne report?

rep = mne.Report(raw)


In [ ]:
import plotly.graph_objects as go
import pandas as pd
from meg_qc.source.universal_plots import make_3d_sensors_trace, keep_unique_locs, switch_names_on_off, QC_derivative
from meg_qc.source.initial_meg_qc import MEG_channels


def plot_sensors_3d_csv(sensors_csv_path: str):

    """
    Plots the 3D locations of the sensors in the raw file. Plot both mags and grads (if both present) in 1 figure. 
    Can turn mags/grads visialisation on and off.
    Separete channels into brain areas by color coding.


    Parameters
    ----------
    chs_by_lobe : dict
        A dictionary of channels by ch type and lobe.
    
    Returns
    -------
    qc_derivative : list
        A list of QC_derivative objects containing the plotly figures with the sensor locations.

    """

    df = pd.read_csv(sensors_csv_path, sep='\t')


    #to not rewrite the whole func, just turn the df back into dic of MEG_channels:

    unique_lobes = df['Lobe'].unique().tolist()

    lobes_dict={}
    for lobe in unique_lobes:
        lobes_dict[lobe] = []
        for index, row in df.iterrows():
            if row['Lobe'] == lobe:
                locs = [row[col] for col in df.columns if 'Sensor_location' in col]
                lobes_dict[lobe].append(MEG_channels(name = row['Name'], type = row['Type'], lobe = row['Lobe'], lobe_color = row['Lobe Color'], loc = locs))

    print(lobes_dict)

    traces = []

    if len(lobes_dict)>1: #if there are lobes - we use color coding: one color pear each lobe
        for lobe in lobes_dict:
            ch_locs, ch_names, ch_color, ch_lobe = keep_unique_locs(lobes_dict[lobe])
            traces.append(make_3d_sensors_trace(ch_locs, ch_names, ch_color[0], 10, ch_lobe[0], 'circle', 'top left'))
            #here color and lobe must be identical for all channels in 1 trace, thi is why we take the first element of the list
            # TEXT SIZE set to 10. This works for the "Always show names" option but not for "Show names on hover" option

    else: #if there are no lobes - we use random colors previously assigned to channels, channel names will be used instead of lobe names in make_3d_trace function
        ch_locs, ch_names, ch_color, ch_lobe = keep_unique_locs(lobes_dict[lobe])
        for i, _ in enumerate(ch_locs):
            traces.append(make_3d_sensors_trace([ch_locs[i]], ch_names[i], ch_color[i], 10, ch_names[i], 'circle', 'top left'))


    fig = go.Figure(data=traces)

    fig.update_layout(
        width=900, height=900,
        title={
        'text': 'Sensors positions',
        'y':0.85,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'})
    
    fig.update_layout(
        scene = dict(
        xaxis = dict(visible=False),
        yaxis = dict(visible=False),
        zaxis =dict(visible=False)
        )
    )

    #check_num_channels_correct(chs_by_lobe, 'END_PLOT') #check we didnt change the original dict


    # Add the button to have names show up on hover or always:
    fig = switch_names_on_off(fig)

    fig.update_traces(hoverlabel=dict(font=dict(size=10))) #TEXT SIZE set to 10 again. This works for the "Show names on hover" option, but not for "Always show names" option

    fig.show()
    
    qc_derivative = [QC_derivative(content=fig, name='Sensors_positions', content_type='plotly', description_for_user="Magnetometers names end with '1' like 'MEG0111'. Gradiometers names end with '2' and '3' like 'MEG0112', 'MEG0113'. ")]

    return qc_derivative 

plot_sensors_3d_csv(sensors_csv_path = '/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds003483/derivatives/Meg_QC/sub-009/sub-009_ses-1_task-deduction_run-1_desc-Sensors_meg.tsv')




In [ ]:
import pandas as pd

df = pd.DataFrame({'c1': [10, 11, 12], 'c2': [100, 110, 120]})
df = df.reset_index()  # make sure indexes pair with number of rows

for index, row in df.iterrows():
    print(row['c1'], row['c2'])

print(df)

In [ ]:
Plotting_paths

In [ ]:
import mne
raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/Paris2020/sub-emptyroom/meg/sub-emptyroom_task-Paris5_meg.fif', allow_maxshield=True)
#raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/LeerraumMD2016/sub-emptyroom/meg/sub-emptyroom_task-Magdeburg2_meg.fif')
raw

In [ ]:
#Aligned Wave Shapes:

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# Generate two aligned wave shapes
time = np.linspace(0, 1, 100)
wave1 = np.sin(2 * np.pi * 2 * time)
wave2 = np.sin(2 * np.pi * 2 * time)

# Calculate correlation
correlation = np.correlate(wave1, wave2, mode='same')
corr1 = pearsonr(wave1, wave2)
print(corr1)

# Plot the wave shapes and correlation
plt.subplot(2, 1, 1)
plt.plot(time, wave1)
plt.title('Wave 1')
plt.subplot(2, 1, 2)
plt.plot(time, wave2)
plt.title('Wave 2')
plt.show()


#Misaligned Wave Shapes:

import numpy as np
import matplotlib.pyplot as plt

# Generate two misaligned wave shapes
time = np.linspace(0, 1, 100)
wave1 = np.sin(2 * np.pi * 2 * time)
wave2 = np.sin(2 * np.pi * 2 * (time + 0.15))  # Shifted by 0.2 seconds

# Calculate correlation
correlation = np.correlate(wave1, wave2, mode='same')
corr2 = pearsonr(wave1, wave2)
print(corr2)

# Plot the wave shapes and correlation
plt.subplot(2, 1, 1)
plt.plot(time, wave1)
plt.title('Wave 1')
plt.subplot(2, 1, 2)
plt.plot(time, wave2)
plt.title('Wave 2')
plt.show()

In [ ]:
#Create wave shapes
import plotly.graph_objects as go
import numpy as np

# Define the number of points in each array
num_points = 100

# Create an array of time values
t = np.linspace(0, 2*np.pi, num_points)

# Define the amplitudes for the R-wave shapes
amplitudes = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5]

# Define the maximum time shift in seconds
max_shift = 0.4

# Create five arrays with R-wave shapes, shifted in time
waves = []
for i, amplitude in enumerate(amplitudes):
    # Generate a random time shift within the maximum shift range
    time_shift = np.random.uniform(-max_shift, max_shift)
    
    # Shift the time values
    shifted_t = t + time_shift
    
    # Create the R-wave shape with the shifted time values
    wave = np.exp(-shifted_t) * np.sin(4*shifted_t) * amplitude
    #waves.append(wave)
    waves.append(wave[::-1])


fig = go.Figure()
for i, wave in enumerate(waves):
    fig.add_trace(go.Scatter(x=time, y=wave, name=f'Wave {i+1}'))
fig.show()

In [ ]:
import numpy as np

# Assuming you have two arrays: array1 and array2
array1 = waves[0]
array2 = waves[5]

import numpy as np
import plotly.graph_objects as go
from scipy.signal import find_peaks

# Assuming you have two arrays: array1 and array2

# Find peaks in both arrays
peaks1, _ = find_peaks(array1)
peaks2, _ = find_peaks(array2)

# Calculate the time shift based on the peak positions
time_shift = peaks1[0] - peaks2[0]

# Shift array2 to align with array1
aligned_array2 = np.roll(array2, time_shift)

# Create the figure
fig = go.Figure()

# Add the array1 trace
fig.add_trace(go.Scatter(x=np.arange(len(array1)), y=array1, name='Array 1'))

# Add the array2 trace
fig.add_trace(go.Scatter(x=np.arange(len(array2)), y=array2, name='Array 2'))

# Add the aligned_array2 trace
fig.add_trace(go.Scatter(x=np.arange(len(aligned_array2)), y=aligned_array2, name='Aligned Array 2'))

# Set the layout
fig.update_layout(title='Aligned Arrays using Peak Detection',
                  xaxis_title='Time',
                  yaxis_title='Amplitude')

# Show the figure
fig.show()

In [ ]:
import numpy as np
import plotly.graph_objects as go
from scipy.signal import find_peaks

array1 = waves[0]
array2 = -waves[5]

# Create the figure
fig = go.Figure()

# Add the array1 trace
fig.add_trace(go.Scatter(x=np.arange(len(array1)), y=array1, name='Array 1'))

# Add the array2 trace
fig.add_trace(go.Scatter(x=np.arange(len(array2)), y=array2, name='Array 2'))

# Set the layout
fig.update_layout(title='Aligned Arrays using Peak Detection',
                  xaxis_title='Time',
                  yaxis_title='Amplitude')

# Show the figure
fig.show()


In [ ]:


# Assuming you have two arrays: array1 and array2

# Find peaks in array1
peaks1, _ = find_peaks(array1)

# Initialize variables for best alignment
best_time_shift = 0
best_correlation = -np.inf
best_aligned_array2 = None

# Try aligning array2 in both orientations
for flip in [False, True]:
    # Flip array2 if needed
    #aligned_array2 = np.flip(array2) if flip else array2
    aligned_array2 = -array2 if flip else array2

    # Find peaks in aligned_array2
    peaks2, _ = find_peaks(aligned_array2)

    # Calculate the time shift based on the peak positions
    time_shift = peaks1[0] - peaks2[0]

    # Shift aligned_array2 to align with array1
    aligned_array2 = np.roll(aligned_array2, time_shift)

    # Calculate the correlation between array1 and aligned_array2
    correlation = np.corrcoef(array1, aligned_array2)[0, 1]

    # Update the best alignment if the correlation is higher
    if correlation > best_correlation:
        best_correlation = correlation
        best_time_shift = time_shift
        best_aligned_array2 = aligned_array2

# Create the figure
fig = go.Figure()

# Add the array1 trace
fig.add_trace(go.Scatter(x=np.arange(len(array1)), y=array1, name='Array 1'))

# Add the array2 trace
fig.add_trace(go.Scatter(x=np.arange(len(array2)), y=array2, name='Array 2'))

# Add the best_aligned_array2 trace
fig.add_trace(go.Scatter(x=np.arange(len(best_aligned_array2)), y=best_aligned_array2, name='Aligned Array 2'))

# Set the layout
fig.update_layout(title='Aligned Arrays with Flipped Second Array',
                  xaxis_title='Time',
                  yaxis_title='Amplitude')

# Show the figure
fig.show()

In [ ]:
import numpy as np
from scipy.signal import find_peaks

avg_ecg_epoch_data_nonflipped_limited_to_event = np.array(waves)

max_values=np.max(np.abs(avg_ecg_epoch_data_nonflipped_limited_to_event), axis=1)
print(max_values)
max_values_ind=np.argsort(max_values)[::-1] 
print(max_values_ind)
max_values_ind=max_values_ind[:5]

chosen_5 = (avg_ecg_epoch_data_nonflipped_limited_to_event[max_values_ind])

thresh_lvl_peakfinder = 5


#get the highest peak for every channel:
max_amplitude1 = []
index_of_max_amplitude1=[]
for ch_data in avg_ecg_epoch_data_nonflipped_limited_to_event:

    thresh_mean=(max(ch_data) - min(ch_data)) / thresh_lvl_peakfinder
    peak_locs_pos, _ = find_peaks(ch_data, prominence=thresh_mean)
    peak_locs_neg, _ = find_peaks(-ch_data, prominence=thresh_mean)

    all_peaks = np.concatenate((peak_locs_pos, peak_locs_neg))
    print('all peaks', all_peaks)

    #Find the peak with the maximal amplitude:

    max_amplitude_peak = np.argmax(np.abs(ch_data[all_peaks]))

    #now find the index of this point in the channel data:
    index_of_max_amplitude1.append(all_peaks[max_amplitude_peak])

    print('Index1', index_of_max_amplitude1)

    #now find the magnitude of the data in this point:

    max_amplitude1.append(ch_data[index_of_max_amplitude1[-1]])

    

# find 5 channels which have the highest peaks and get the locations of these peaks:
highest_channels_sorted = np.argsort(max_amplitude1)[::-1] 
print(highest_channels_sorted)
max_ind_of_chosen_5=highest_channels_sorted[:5]

print(max_ind_of_chosen_5)


In [ ]:
index_of_max_amplitude2=[]
for ch_data in max_ind_of_chosen_5:

    thresh_mean=(max(ch_data) - min(ch_data)) / thresh_lvl_peakfinder
    peak_locs_pos, _ = find_peaks(ch_data, prominence=thresh_mean)
    peak_locs_neg, _ = find_peaks(-ch_data, prominence=thresh_mean)

    all_peaks = np.concatenate((peak_locs_pos, peak_locs_neg))
    print('all peaks', all_peaks)

    #Find the peak with the maximal amplitude:

    max_amplitude_peak = np.argmax(np.abs(ch_data[all_peaks]))


    #6. Output the index of the point with the maximal amplitude:

    index_of_max_amplitude1.append(all_peaks[max_amplitude_peak])
    print('Index1', index_of_max_amplitude1)

    if len(all_peaks)>1:
        #7. Now find the second largest peak:
        all_peaks_without_max = np.delete(all_peaks, max_amplitude_peak)

        print('no max', all_peaks_without_max)

        max_amplitude_peak = np.argmax(np.abs(ch_data[all_peaks_without_max]))


        #6. Output the index of the point with the maximal amplitude:

        index_of_max_amplitude2.append(all_peaks_without_max[max_amplitude_peak])
        print('Index2', index_of_max_amplitude2)
        
    else:
        index_of_max_amplitude2.append(np.nan)

mean_index_of_max_amplitude1 = np.nanmean(index_of_max_amplitude1)

# If in more than a half of cases there was no second biggest peak found, skip it and assign t) as first peak:
non_zero_count = np.count_nonzero(index_of_max_amplitude2)
percentage = (non_zero_count/len(index_of_max_amplitude2)) * 100

if percentage < 50:
    t0_peak = int(mean_index_of_max_amplitude1)
else:
    mean_index_of_max_amplitude2 = np.nanmean(index_of_max_amplitude2)
    #Now out of them set the first peak (according to time) as t0.
    t0_peak = int(np.nanmin([mean_index_of_max_amplitude1, mean_index_of_max_amplitude2]))


print('mean_ind1', mean_index_of_max_amplitude1)
print('mean_ind2', mean_index_of_max_amplitude2)


print(t0_peak)



In [ ]:
import numpy as np

arr = np.array([5, 2, 9, 1, 7, 3])

# Get the indices that would sort the array in ascending order
sorted_indices = np.argsort(arr)

# Index of the largest value
largest_index = sorted_indices[-1]

# Index of the second largest value
second_largest_index = sorted_indices[-2]

print("Index of the largest value:", largest_index)
print("Index of the second largest value:", second_largest_index)

In [ ]:
import mne
from IPython.display import display

# Load the MEG data
raw=mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds004107/sub-mind002/ses-01/meg/sub-mind002_ses-01_task-auditory_meg.fif', preload=True)

display(raw)


In [ ]:

# Define the EOG channel names
eog_channels = ['EOG 061', 'EOG 062']

# extract the data of 2 EOG channels
eog_data = raw.copy().pick_channels(eog_channels).get_data()

print(eog_data)


In [ ]:

# Plot the data with plotly:

import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Create a figure with two subplots
fig = make_subplots(rows=2, cols=1)

x_values = raw.times

# Add a trace for the first subplot
fig.add_trace(go.Scatter(x=x_values, y=eog_data[0], mode='lines', name='EOG 1'), row=1, col=1)

# Add a trace for the second subplot
fig.add_trace(go.Scatter(x=x_values, y=eog_data[1], mode='lines', name='EOG 2'), row=2, col=1)

# Update the layout
fig.update_layout(title='EOG Data', xaxis_title='Time (s)', yaxis_title='Amplitude')

# Show the figure
fig.show()



In [ ]:
import numpy as np

# create two arrays
array1 = np.array([1, 2, 3, 4, 5])
array2 = np.array([6, 7, 8, 9, 10])

# stack the arrays horizontally
stacked = np.stack((array1, array2), axis=0)

display(stacked)

# calculate the covariance matrix
covariance_matrix = np.cov(stacked)

print(covariance_matrix)


In [ ]:
import mne
import numpy as np
from scipy.signal import find_peaks

# Load the MEG data
raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds003703/sub-a68d5xp5/meg/sub-a68d5xp5_task-listeningToSpeech_run-01_meg.fif')
#raw=mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/openneuro/ds004107/sub-mind002/ses-01/meg/sub-mind002_ses-01_task-auditory_meg.fif', preload=True)

# Select the EOG channels
eog_channels = mne.pick_types(raw.info, meg=False, eeg=False, stim=False, eog=True)

# Get the names of the EOG channels
eog_channel_names = [raw.ch_names[ch] for ch in eog_channels]

print('EOG channel names:', eog_channel_names)

eog_events = mne.preprocessing.find_eog_events(raw)

In [ ]:

picks_ECG = mne.pick_types(raw.info, ecg=True)
ecg_ch_name = [raw.info['chs'][name]['ch_name'] for name in picks_ECG]

arr=raw.get_data(picks=ecg_ch_name)[0] 
height = np.mean(arr) + 1 * np.std(arr)
fs=raw.info['sfreq']
peaks, _ = find_peaks(arr, height=height, distance=round(0.5 * fs)) #assume there are no peaks within 0.5 seconds from each other.
ecg_events = peaks/fs

# Define the time window of interest
time_window = [0.2, 0.2]  # in seconds
tmin=-0.2
tmax=0.2

# Convert time window to samples
sfreq = 1000  # sampling frequency of your data
time_window_samples = np.round(np.array(time_window) * sfreq).astype(int)
print('samples', time_window_samples)

# Initialize an empty array to store the extracted epochs
epochs = np.zeros((len(peaks), int((tmax-tmin)*sfreq)))

print('HERE')
print(arr)
print(epochs)

# Loop through each ECG event and extract the corresponding epoch
for i, event in enumerate(peaks):
    start = event - time_window_samples[0]
    start = np.round(event + tmin*sfreq).astype(int)
    end = event + time_window_samples[1]
    end= np.round(event + tmax*sfreq).astype(int)
    epochs[i, :] = arr[start:end]

#average all epochs:
avg_ecg=np.mean(epochs, axis=0)

#print average ecg with plotly:
import plotly.graph_objects as go

fig = go.Figure()
#create time vector based on time window and sampling frequency:
times= np.arange(tmin, tmax, 1/sfreq)
fig.add_trace(go.Scatter(x=times, y=avg_ecg, mode='lines', name='ECG'))
fig.show()




In [ ]:

# Detect the R-wave peaks in the filtered ECG channel data
r_peaks, ch_ecg, pulse, ecg_data_rec = mne.preprocessing.find_ecg_events(raw, return_ecg=True)
print(ecg_data_rec)

#plot the ECG data with plotly:
import plotly.graph_objects as go

times=[t for t in range(len(ecg_data_rec[0]))]
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=ecg_data_rec[0], mode='lines', name='ECG'))
fig.update_layout(title='ECG data', xaxis_title='Time (s)', yaxis_title='ECG (mV)')
fig.show()

# Calculate the time difference between each R-wave peak and the first R-wave peak
r_wave_epochs = (r_peaks - r_peaks[0]) / raw.info['sfreq']
print('r_wave_epochs', r_wave_epochs)

# Calculate the average R-wave epoch
avg_r_wave_epoch = np.mean(r_wave_epochs)
print('avg_r_wave_epoch', avg_r_wave_epoch)

if ecg_ch_name:
    # Extract the ECG channel data
    ecg_data, times = raw.get_data(ecg_ch_name, return_times=True)
    ecg_data2=ecg_data_rec
else:
    ecg_data=ecg_data_rec



# Use the average R-wave epoch to extract a segment of data from the ECG channel
avg_r_wave_data = ecg_data[:, int(avg_r_wave_epoch * raw.info['sfreq']) : int((avg_r_wave_epoch + 0.2) * raw.info['sfreq'])]
avg_r_wave_data2 = ecg_data2[:, int(avg_r_wave_epoch * raw.info['sfreq']) : int((avg_r_wave_epoch + 0.2) * raw.info['sfreq'])]

#plot the average R-wave epoch with plotly:
import plotly.graph_objects as go

times=[t/raw.info['sfreq'] for t in range(len(avg_r_wave_data[0]))]
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=avg_r_wave_data[0], mode='lines', name='ECG'))
fig.add_trace(go.Scatter(x=times, y=avg_r_wave_data2[0], mode='lines', name='ECG2'))
fig.update_layout(title='Average R-wave epoch', xaxis_title='Time (s)', yaxis_title='ECG (mV)')
fig.show()

raw

In [ ]:
print([t/raw.info['sfreq'] for t in range(len(avg_r_wave_data[0]))])

In [ ]:
ecg_data
avg_r_wave_epoch * raw.info['sfreq']
r_peaks[0]

In [ ]:
avg_r_wave_data = ecg_data[:, int(avg_r_wave_epoch * raw.info['sfreq']) : int((avg_r_wave_epoch + 0.2) * raw.info['sfreq'])]
avg_r_wave_data2 = ecg_data2[:, int(avg_r_wave_epoch * raw.info['sfreq']) : int((avg_r_wave_epoch + 0.2) * raw.info['sfreq'])]

#plot the average R-wave epoch with plotly:
import plotly.graph_objects as go

times=[t/raw.info['sfreq'] for t in range(len(avg_r_wave_data[0]))]
fig = go.Figure()
fig.add_trace(go.Scatter(x=times, y=avg_r_wave_data[0], mode='lines', name='ECG'))
fig.add_trace(go.Scatter(x=times, y=avg_r_wave_data2[0], mode='lines', name='ECG2'))
fig.update_layout(title='Average R-wave epoch', xaxis_title='Time (s)', yaxis_title='ECG (mV)')
fig.show()

In [ ]:
import numpy as np
from scipy.stats import pearsonr

# Generate two waves
wave1 = np.array([1, 2, 3, 4, 5])
wave2 = np.array([2, 4, 6, 8, 10])

# Calculate the Pearson correlation coefficient and p-value
corr_coef, p_value = pearsonr(wave1, wave2)

# Print the results
print("Pearson correlation coefficient:", corr_coef)
print("p-value:", p_value)


#plot both waves with plotly:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y=wave1, mode='lines', name='wave1'))
fig.add_trace(go.Scatter(y=wave2, mode='lines', name='wave2'))
fig.show()


In [ ]:
import random

lobe_colors = {
        'Left Frontal': '#1f77b4',
        'Right Frontal': '#ff7f0e',
        'Left Temporal': '#2ca02c',
        'Right Temporal': '#9467bd',
        'Left Parietal': '#e377c2',
        'Right Parietal': '#d62728',
        'Left Occipital': '#bcbd22',
        'Right Occipital': '#17becf'}

print(random.choice(list(lobe_colors.values())))

In [ ]:
from scipy.ndimage import gaussian_filter
import numpy as np

# Generate some noisy wave data
x = np.linspace(0, 2*np.pi, 100)
y = np.sin(x) + np.random.normal(0, 0.5, 100)

# Apply Gaussian smoothing with a sigma of 2
y_smooth = gaussian_filter(y, sigma=4)

# Plot the original and smoothed waves
import matplotlib.pyplot as plt
plt.plot(x, y, label='Noisy wave')
plt.plot(x, y_smooth, label='Smoothed wave')
plt.legend()
plt.show()


In [ ]:
import plotly.graph_objs as go
import pandas as pd

# create sample data
df = pd.DataFrame({'values': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]}, index=['A']*10)
df = df.T
print(df)

# create box plot trace
box_trace = go.Box(x=df.iloc[0], orientation='h')
#box_trace = go.Box(x=df['values'], y=df.index, orientation='h', name='')

fig = go.Figure(data=box_trace)

for col in df.columns:
    fig.add_trace(go.Scatter(x=df[col], name=col))

# for v in df['values']:
#     #fig.add_trace(go.Scatter(x=df['values'], y=df.index, mode='markers', marker=dict(size=5, color='yellow'), name='Scatter Plot', hovertext=df.index))
#     fig.add_trace(go.Scatter(x=[v], y=['A'], mode='markers', marker=dict(size=5, color='yellow'), name='Scatter Plot', hovertext=df.index))

# plot figure
fig.show()


In [ ]:
import plotly.graph_objs as go

fig = go.Figure()

# Add box plot trace
fig.add_trace(go.Box(x=[1, 2, 3, 4, 5]))

# Add horizontal line at y=0
fig.update_layout(
    shapes=[
        dict(
            type='line',
            yref='y',
            y0=0,
            y1=0,
            xref='paper',
            x0=0,
            x1=1
        )
    ]
)

fig.show()


In [ ]:
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Create example dataset
np.random.seed(123)
std_val = pd.DataFrame({'Group': ['A', 'A', 'B', 'B', 'C', 'C'],
                     'Value': np.random.normal(size=6)})

# Create box plot with custom marker colors
fig = go.Figure()
fig.add_trace(go.Box(x=std_val['Group'], y=std_val['Value'], name='Value',
                     marker=dict(color='red')))

# Update layout
fig.update_layout(title='Box plot with custom marker colors',
                  xaxis_title='Group', yaxis_title='Value')

# Show the plot
fig.show()

In [ ]:
import plotly.graph_objs as go

# create a box plot with custom marker color
trace = go.Box(
    y=[1, 2, 3, 4, 5],
    marker=dict(
        color='blue'
    )
)

# create a figure and add the box plot trace
fig = go.Figure(data=[trace])

# show the figure
fig.show()


In [ ]:
#This here is to save all the average ECG/EOG data into a pickle file, so I can test difefrent wave detection algorythms on them without running the pipeline again

import pickle 
import plotly.graph_objects as go
import numpy as np


# open a file in write binary mode
with open("avg_ecg.pkl", "wb") as f:
    # dump the list of objects into the file
    pickle.dump(avg_ecg, f)

with open("avg_eog.pkl", "wb") as f:
    # dump the list of objects into the file
    pickle.dump(avg_eog, f)




In [ ]:
#This here is to open the pickle files from above and plot the data

import pickle 
import plotly.graph_objects as go
import numpy as np

# open the file in read binary mode
with open("avg_ecg0.pkl", "rb") as f:
    # load the list of objects from the file
    eog_list = pickle.load(f)

print(eog_list[0])

sfreq=1000
t = np.round(np.arange(-0.4, 0.4+1/sfreq, 1/sfreq), 3) #yes, you need to round
fig0=go.Figure()
for x in range(0, len(eog_list)):
    fig_temp=eog_list[x].plot_epoch_and_peak(t, 'Channels affected by ECG artifact: ', 'mag', fig0)
    for trace in fig_temp['data']:
        fig0.add_trace(trace)

fig0.update_layout(
    yaxis = dict(
            showexponent = 'all',
            exponentformat = 'e')) 
fig0.show()


In [ ]:
# same for EOG

import pickle 
import plotly.graph_objects as go
import numpy as np
from copy import deepcopy
from scipy.ndimage import gaussian_filter


# open the file in read binary mode
with open("avg_eog0.pkl", "rb") as f:
    # load the list of objects from the file
    eog_list = pickle.load(f)

sfreq=1000
t = np.round(np.arange(-0.4, 0.4+1/sfreq, 1/sfreq), 3) #yes, you need to round
fig0=go.Figure()
for x in range(0, len(eog_list)):
    fig_temp=eog_list[x].plot_epoch_and_peak(t, 'Channels affected by ECG artifact: ', 'mag')
    for trace in fig_temp['data']:
        fig0.add_trace(trace)

fig0.update_layout(
    yaxis = dict(
            showexponent = 'all',
            exponentformat = 'e')) 
fig0.show()

#Now apply the gaussia filter to each trace and plot result in the same figure:
fig0_new=deepcopy(fig0)
for trace in fig0_new['data']:
    y=trace['y']
    y_smooth = gaussian_filter(y, sigma=10)
    trace['y']=y_smooth

fig0_new.show()

In [ ]:
import mne
raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)

In [ ]:
#show sensor posiions using mne:

import mne

raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)

mne.viz.plot_sensors(raw.info, kind='topomap', ch_type='grad', show_names=True, ch_groups='position')


In [ ]:
#PLOT SENSORS IN 2D with plotly
import numpy as np
import plotly.graph_objects as go
import mne

raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)


mag_ch_names = raw.copy().pick_types(meg='mag').ch_names if 'mag' in raw else None
grad_ch_names = raw.copy().pick_types(meg='grad').ch_names if 'grad' in raw else None
channels_objs = {'mag': mag_ch_names, 'grad': grad_ch_names}

# Get the sensor locations
sensor_locs = raw.info['chs']
#print(sensor_locs)
#coords_mag=[loc['loc'][:2] for loc in sensor_locs]
coords_mag=[loc['loc'] for loc in sensor_locs if loc['ch_name'] in mag_ch_names]
#print(len(coords), coords)
print(len(coords_mag), coords_mag)

x = [r[0] for r in coords_mag]
y = [r[1] for r in coords_mag]
#x, y, z = [loc['loc'][:3] for loc in sensor_locs]
names = [loc['ch_name'] for loc in sensor_locs if loc['ch_name'] in mag_ch_names]
kinds= [loc['kind'] for loc in sensor_locs]
print(kinds)

# Create a scatter plot of the sensor locations
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='markers', text=names))

fig.update_layout(
    autosize=False,
    width=1000,
    height=1000)

# Set the plot title and axis labels
fig.update_layout(title='MEG Sensor Locations', xaxis_title='X', yaxis_title='Y')

# Add a circle shape to the plot to show the position of the head
fig.update_layout(
    shapes=[
        dict(
            type='circle',
            xref='x',
            yref='y',
            x0=-0.1,
            y0=-0.1,
            x1=0.1,
            y1=0.12,
            line=dict(color='red', width=2),
            opacity=0.5
        ),
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=[0, -0.02],
            y0=[0.1, 0.08],
            line=dict(color='black', width=2)
        ),
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=[0, 0.02],
            y0=[0.1, 0.08],
            line=dict(color='black', width=2)
        ),
        dict(
            type='line',
            xref='x',
            yref='y',
            x0=[-0.02, 0.02],
            y0=[0.08, 0.08],
            line=dict(color='black', width=2))
])



# Show the plot 
fig.show()

In [ ]:
raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)

#PLOT 3 D

def switch_names_on_off(fig):
    # Define the buttons
    buttons = [
    dict(label='Show channel names when hovering',
         method='update',
         args=[{'mode': 'markers'}]),
    dict(label='Always show channel names',
         method='update',
         args=[{'mode': 'markers+text'}])
    ]

    # Add the buttons to the layout
    fig.update_layout(updatemenus=[dict(type='buttons',
                                        showactive=True,
                                        buttons=buttons)])

    return fig



# Extract the sensor locations and names for magnetometers
mag_locs = raw.copy().pick_types(meg='mag').info['chs']
mag_pos = [ch['loc'][:3] for ch in mag_locs]
mag_names = [ch['ch_name'] for ch in mag_locs]

# Create the magnetometer plot with markers only

mag_fig = go.Figure(data=[go.Scatter3d(x=[pos[0] for pos in mag_pos],
                                       y=[pos[1] for pos in mag_pos],
                                       z=[pos[2] for pos in mag_pos],
                                       mode='markers',
                                       marker=dict(size=5),
                                       text=mag_names,
                                       hovertemplate='%{text}')],
                                       layout=go.Layout(width=1000, height=1000))

mag_fig.update_layout(title='Magnetometers')

mag_fig = switch_names_on_off(mag_fig)
mag_fig.show()



# Extract the sensor locations and names for gradiometers
grad_locs = raw.copy().pick_types(meg='grad').info['chs']
grad_pos = [ch['loc'][:3] for ch in grad_locs]
grad_names = [ch['ch_name'] for ch in grad_locs]

#since grads have 2 sensors located in the same spot - need to put their names together to make pretty plot labels:

grad_pos_together = []
grad_names_together = []

for i in range(len(grad_pos)-1):
    if all(x == y for x, y in zip(grad_pos[i], grad_pos[i+1])):
        grad_pos_together += [grad_pos[i]]
        grad_names_together += [grad_names[i]+', '+grad_names[i+1]]
    else:
        pass


# Add both sets of gradiometer positions to the plot:
grad_fig = go.Figure(data=[go.Scatter3d(x=[pos[0] for pos in grad_pos_together],
                                        y=[pos[1] for pos in grad_pos_together],
                                        z=[pos[2] for pos in grad_pos_together],
                                        mode='markers',
                                        marker=dict(size=5),
                                        text=grad_names_together,
                                        hovertemplate='%{text}')],
                                        layout=go.Layout(width=1000, height=1000))

grad_fig.update_layout(title='Gradiometers')


# Add the button to have names show up on hover or always:
grad_fig = switch_names_on_off(grad_fig)

# Show the plots

grad_fig.show()


In [ ]:

# MUSCLE ARIFACTS IN EMPTYROOM DATA:
# Discussed with Andreas:
# We can see very high muscle scores at the very beginning and end of the empty room recording
# Are these real muscle artifacts or filtering errors?
# Cut out 1st second of the data where they are visible.
# make Fourier transform of the 1st second and see if there are high amplitudes visible for the muscle frequencies - nope 
# most likely this is filtering.
# next, follow the MNE steps for muscle artifact detection: they use first filtering at 11--140 hz, then Hilbert
# plotted raw data after the applied filter, and Hilbert - see cut artifacts in the beginning and end. (WJY arw we sure it  s not hilbert?)
# then, tried to only filter - very noisy data. but most likely the filtering is the source. Because of the cut-off in the beginning and end.
# Solutions: zero padding in the beginning and end before filtering, which will be cut off after. But may still create a jump while filtering and keep the artifact.
# Better: add 2s of dummy data at the beginning and end of the recording, and then crop it out (the data added should be mirrored). This will not create a jump in the filtering.
# Tried

# Problem found! 2 problems: 
# 1st: The main artifact is actually introduced by filtering power lines. filtering the data at 150 Hz (harmonics) clearly creates this artifact.
# Removed that and any other filtering over the range of muscle freqs, since we don't need them anyways. (over 140 Hz)
# 2nd: Still some artifact is present in the beginning and end of the recording. For this attach mirrored data on both ends.
# Then detect muscle, then cut the resulting scores away for the attached period.
# There will be still some very minimal artifact at the beginning/end of this attachment - probably due to the attachment itself: the mirrored data is still not a normally shaped signal.
# See example in cell above of how all 3 option look: oroginal, with attached data and with attached and cut away.

import numpy as np
import plotly.graph_objects as go
import mne

raw = mne.io.read_raw_fif('/Volumes/M2_DATA/MEG_QC_stuff/data/Jochem/LeerraumAarhus2017/sub-emptyroom/meg/sub-emptyroom_task-Aarhus_meg.fif', preload=True)

#%matplotlib qt
raw_first = raw.copy().crop(tmin=0, tmax=1)
#raw_first.plot()


std_val=raw_first.get_data()

window = np.hanning(std_val.shape[-1])*std_val

window

freqs = np.fft.rfftfreq(window.shape[-1], 1/raw.info['sfreq'])

components = np.fft.fft(window, axis=-1)

components.shape

fig = go.Figure()
for ch in range(15, 250):
    fig.add_trace(go.Scatter(x=freqs, y = np.abs(components[ch, 0:500])))
fig.add_trace(go.Scatter(x=freqs, y = np.abs(components[0, 0:500])))
#from mne annotate_muscle_zscore:
from scipy.stats import zscore
from scipy.ndimage import label

filter_freq=(110, 140)
legend_category = 'mag'

raw_copy = raw_first.copy()
raw_copy.load_data()

if legend_category is None:
    raw_ch_type = raw_copy.get_channel_types()
    if 'mag' in raw_ch_type:
        legend_category = 'mag'
    elif 'grad' in raw_ch_type:
        legend_category = 'grad'
    elif 'eeg' in raw_ch_type:
        legend_category = 'eeg'
    else:
        raise ValueError('No M/EEG channel types found, please specify a'
                            ' ch_type or provide M/EEG sensor data')

if legend_category in ('mag', 'grad'):
    raw_copy.pick_types(meg=legend_category, ref_meg=False)
else:
    legend_category = {'meg': False, legend_category: True}
    raw_copy.pick_types(**legend_category)

#raw_copy.filter(filter_freq[0], filter_freq[1], fir_design='firwin',
#                pad="reflect_limited")

hilb_applied=raw_copy.apply_hilbert(envelope=True)
hilb_applied.plot()



In [ ]:
import numpy as np

# Load the list of values into a NumPy array
values = np.array([1, 2, 1.8, 2.5, 3, 3.5, 4, 3.8, 5, 4, 2, 2.1, 1, 0, 2, 4, 6])



import numpy as np
from scipy.signal import butter, filtfilt, savgol_filter, find_peaks

import plotly.graph_objects as go
import numpy as np

# Generate a noisy wave shape
t = np.linspace(0, 10, 1000)
y = np.sin(t) + 0.5*np.random.randn(len(t))

data = np.random.randn(1000) #no wave shape
# Load the noisy wave data into a NumPy array
wave_data = data

# Create a Plotly figure
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=wave_data, mode='lines', name='Noisy Wave'))
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Noisy Wave Shape')
fig.show()

# Apply a low-pass filter to remove high-frequency noise
b, a = butter(5, 0.1, 'low')
filtered_data = filtfilt(b, a, wave_data)

# plot filtered data
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=filtered_data, mode='lines', name='Filtered Wave'))
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Filtered Wave Shape')
fig.show()

# Apply a Savitzky-Golay filter to further reduce noise and extract the underlying wave shape
#smoothed_data = savgol_filter(wave_data, window_length=int(len(wave_data)/4), polyorder=3)
smoothed_data = savgol_filter(data, window_length=100, polyorder=3)

# Identify the shape of the wave using peak detection or curve fitting
# For example, you can use the `scipy.signal.find_peaks` function to detect peaks in the smoothed data
#peaks, _ = find_peaks(smoothed_data, height=0.5*np.max(smoothed_data))
peaks, _ = find_peaks(smoothed_data)

# plot smoothed data
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=smoothed_data, mode='lines', name='Smoothed Wave'))
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Smoothed Wave Shape')
fig.add_trace(go.Scatter(x=t[peaks], y=smoothed_data[peaks], mode='markers', name='Peaks'))
fig.show()



In [ ]:
import plotly.graph_objects as go
import numpy as np
from scipy.signal import find_peaks

# Generate a noisy wave shape
t = np.linspace(0, 10, 1000)
y = np.sin(t) + 0.5*np.random.randn(len(t))

# Find the peaks in the wave
peaks, _ = find_peaks(y)

# Count the number of peaks
num_peaks = len(peaks)

# Create a Plotly figure
fig = go.Figure()

# Add the noisy wave shape to the figure
fig.add_trace(go.Scatter(x=t, y=y, mode='lines', name='Noisy Wave'))

# Add the peaks to the figure
fig.add_trace(go.Scatter(x=t[peaks], y=y[peaks], mode='markers', name='Peaks'))

# Add axis labels and a title
fig.update_layout(xaxis_title='Time', yaxis_title='Amplitude', title='Noisy Wave Shape')

# Show the figure and print the number of peaks
fig.show()
print(f'The wave has {num_peaks} crest(s).')

In [ ]:
import mne
picks_EOG = mne.pick_types(raw.info, eog=True)
eog_ch_name = [raw.info['chs'][name]['ch_name'] for name in picks_EOG]
eog_ch_name

In [ ]:
one_psd = [2.93686870e-12, 5.37336497e-13, 2.34749324e-13, 1.70403629e-13
, 1.42868936e-13, 1.10614848e-13, 1.01586902e-13, 9.41699507e-14
, 8.41904711e-14, 7.56254639e-14, 6.98933286e-14, 6.47116338e-14
, 5.37107007e-14, 5.42174045e-14, 4.78692577e-14, 4.36476164e-14
, 3.69272073e-14, 3.81479068e-14, 4.07614720e-14, 3.71683505e-14
, 3.74843265e-14, 3.57210926e-14, 3.99173535e-14, 4.87143053e-14
, 4.20066645e-14, 3.84896719e-14, 3.13482998e-14, 2.86289627e-14
, 2.82586165e-14, 2.71780036e-14, 2.48692250e-14, 2.71251350e-14
, 2.79561808e-14, 2.72047767e-14, 2.79637330e-14, 2.55955578e-14
, 2.53291180e-14, 2.01500680e-14, 2.12080778e-14, 2.18529602e-14
, 2.12775368e-14, 2.14334140e-14, 2.18751322e-14, 1.97884378e-14
, 1.89952388e-14, 1.79404586e-14, 2.01555022e-14, 2.21105500e-14
, 1.87897255e-14, 1.95585625e-14, 2.07067862e-14, 2.15786185e-14
, 1.78539522e-14, 1.89461927e-14, 1.83714513e-14, 1.91272285e-14
, 1.81790918e-14, 1.51935998e-14, 1.55331746e-14, 1.37627320e-14
, 1.37333973e-14, 1.47261781e-14, 1.35323358e-14, 1.23234074e-14
, 1.26296023e-14, 1.39794705e-14, 1.32391885e-14, 1.33172509e-14
, 1.40752537e-14, 1.35291881e-14, 1.46771014e-14, 1.57039580e-14
, 1.76870590e-14, 2.11409680e-14, 2.66470647e-14, 2.09066429e-14
, 1.68226688e-14, 1.63034232e-14, 1.32317697e-14, 1.20372472e-14
, 1.10395275e-14, 1.17336558e-14, 1.12817157e-14, 1.31068881e-14
, 1.36940739e-14, 1.48016686e-14, 1.35999052e-14, 1.56644411e-14
, 1.51726149e-14, 1.95274934e-14, 1.84669709e-14, 1.89443054e-14
, 1.82544652e-14, 1.92617658e-14, 1.80902967e-14, 2.17239287e-14
, 2.67917043e-14, 4.45194367e-14, 2.01857457e-12, 4.40594585e-12
, 1.74602155e-12, 4.30562941e-14, 2.53461498e-14, 1.87278757e-14
, 1.54232144e-14, 1.67924147e-14, 1.22749773e-14, 1.25017017e-14
, 1.22475994e-14, 1.02921463e-14, 1.07700101e-14, 1.02658035e-14
, 9.54949869e-15, 9.84280694e-15, 8.88745310e-15, 9.02206922e-15
, 8.47210049e-15, 8.64491709e-15, 1.32254861e-14, 1.89573615e-14
, 1.23079196e-14, 8.62994931e-15, 8.12535185e-15, 8.01035318e-15
, 7.53220890e-15, 8.02056256e-15, 7.90231409e-15, 7.63270083e-15
, 7.93212379e-15, 7.28368608e-15, 7.59772607e-15, 7.26136429e-15
, 7.86504197e-15, 7.36256360e-15, 7.02847343e-15, 7.08620266e-15
, 6.86068169e-15, 7.21792433e-15, 7.28674098e-15, 6.88181371e-15
, 6.78751472e-15, 6.59002904e-15, 6.74850515e-15, 6.53743454e-15
, 6.70834535e-15, 6.72520433e-15, 6.78371437e-15, 6.70420118e-15
, 6.97442862e-15, 7.26767528e-15, 6.79388360e-15, 6.83101277e-15
, 6.90684197e-15, 6.45716620e-15, 6.66190889e-15, 6.49304182e-15
, 6.38068712e-15, 6.29160702e-15, 5.92354089e-15, 6.33890242e-15
, 6.33787606e-15, 5.76688121e-15, 6.31821916e-15, 6.34536916e-15
, 6.51250512e-15, 6.43164190e-15, 6.46530769e-15, 6.44724883e-15
, 7.48305304e-15, 7.50925230e-15, 6.28419317e-15, 6.33908319e-15
, 5.86954984e-15, 6.54561206e-15, 6.08872456e-15, 6.40874736e-15
, 5.95870142e-15, 6.13488554e-15, 5.83721527e-15, 5.87793931e-15
, 5.81207088e-15, 5.98748087e-15, 5.94551525e-15, 5.75415575e-15
, 5.66968278e-15, 6.11006036e-15, 5.72066372e-15, 5.96629716e-15
, 5.80372053e-15, 5.75583336e-15, 5.84628922e-15, 5.63642362e-15
, 5.34942930e-15, 5.75920960e-15, 6.05337029e-15, 6.61372576e-15
, 7.14210116e-15, 6.94968538e-15, 1.85742697e-14, 3.52090532e-14
, 1.48785528e-14, 6.23577963e-15, 5.66229647e-15, 5.39212323e-15
, 5.32890121e-15, 5.54967559e-15, 5.29485491e-15, 5.65665900e-15
, 5.31337965e-15, 5.30139224e-15, 5.21434237e-15, 5.61739646e-15
, 5.62673191e-15, 5.68441483e-15, 5.43332729e-15, 5.34563989e-15
, 5.69510011e-15, 6.43038706e-15, 5.52069097e-15, 5.22891308e-15
, 5.06513758e-15, 5.15715319e-15, 5.32484298e-15, 5.37071225e-15
, 5.24099974e-15, 5.14413780e-15, 5.05322799e-15, 5.27277366e-15
, 5.17209094e-15, 5.19895605e-15, 5.04662049e-15, 5.13492402e-15
, 5.39573281e-15, 5.13639899e-15, 5.29696474e-15, 5.29749076e-15
, 5.23187737e-15, 5.14179424e-15, 1.44011463e-14, 2.29753019e-14
, 8.85041560e-15, 5.16398069e-15, 5.09075672e-15, 5.06681957e-15
, 5.17284653e-15, 4.99688083e-15, 5.01988585e-15, 5.07952302e-15
, 4.97188381e-15, 5.17733558e-15, 4.97124292e-15, 4.96910679e-15
, 4.96283207e-15, 5.07193856e-15, 4.80712108e-15, 4.97630935e-15
, 4.93727883e-15, 4.84091247e-15, 5.07370238e-15, 4.76459850e-15
, 4.86678392e-15, 5.03907955e-15, 4.91645908e-15, 4.99691785e-15
, 4.81326372e-15, 5.56398292e-15, 5.40286001e-15, 4.91762834e-15
, 4.96042200e-15, 4.86125369e-15, 5.04306529e-15, 4.89229744e-15
, 4.93924928e-15, 4.91889752e-15, 4.92336366e-15, 4.91476256e-15
, 4.90731383e-15, 4.79751988e-15, 4.96181659e-15, 5.04353794e-15]

#plot the data with ploty:
import numpy as np
import plotly.graph_objects as go
from scipy.signal import find_peaks

freqs = [i/2 for i in range(0, 280)]
prominence_lvl_pos = 50
prominence_pos=(max(one_psd) - min(one_psd)) / prominence_lvl_pos
noisy_freqs_indexes, _ = find_peaks(one_psd, prominence=prominence_pos)
noisy_freqs_indexes = [int(i) for i in noisy_freqs_indexes]

for i in range(0,2):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=freqs, y=one_psd, name='psd'))
    fig.update_layout(title=' PSD', xaxis_title='Frequency', yaxis_title='Amplitude',
            yaxis = dict(
            showexponent = 'all',
            exponentformat = 'e'))

    fig.add_trace(go.Scatter(x=[freqs[noisy_freqs_indexes[0]]], y=[one_psd[noisy_freqs_indexes[0]]], mode='markers', name='peaks'))

    if i == 0:
        fig.update_yaxes(type="log")

    fig.show()


In [ ]:
df_head_pos


In [ ]:
bool('False')

#convert "false" to boolean:
import ast
t = ast.literal_eval('False')
t
